In [56]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [57]:
#Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:


In [58]:
import getpass
password = getpass.getpass()

········


In [59]:
# Check the number of rentals for each customer for May

In [60]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)
may_data = pd.read_sql_query('select customer_id, count(rental_id) as rental_count from sakila.rentals_may group by customer_id order by rental_count desc', engine)
may_data

,customer_id,rental_count
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6
...,...,...
515,431,1
516,351,1
517,10,1
518,136,1


In [61]:
# Check the number of rentals for each customer for June

In [62]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/bank'
engine = create_engine(connection_string)
june_data = pd.read_sql_query('select customer_id, count(rental_id) as rental_count from sakila.rentals_june group by customer_id order by rental_count desc', engine)
june_data

,customer_id,rental_count
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9
...,...,...
585,425,1
586,412,1
587,335,1
588,22,1


In [63]:
# Write a function that checks if customer borrowed more or less films in the month of June as compared to May.

In [64]:
may_data = pd.read_sql_query('select customer_id, count(rental_id) as rental_count from sakila.rentals_may group by customer_id order by rental_count desc', engine)

# Renamaming the columns
may_data = may_data.rename(columns={'rental_count': 'rentals_may'})


june_data = pd.read_sql_query('select customer_id, count(rental_id) as rental_count from sakila.rentals_june group by customer_id order by rental_count desc', engine)

# Renamaming the columns
june_data = june_data.rename(columns={'rental_count': 'rentals_june'})

# Merging the dfs
final_data = pd.merge(may_data, june_data, on='customer_id', how='outer')



In [65]:
for index, row in final_data.iterrows():
    may_rentals = row['rentals_may']
    june_rentals = row['rentals_june']

    if pd.isna(may_rentals):
        final_data.loc[index, 'result'] = 'nan alert'
    elif may_rentals > june_rentals:
        final_data.loc[index, 'result'] = 'June Wins!'
    elif may_rentals < june_rentals:
        final_data.loc[index, 'result'] = 'May Wins!'
    else:
        final_data.loc[index, 'result'] = 'It\'s a tie!'

final_data

,customer_id,rentals_may,rentals_june,result
0,197,8.0,8.0,It's a tie!
1,506,7.0,5.0,June Wins!
2,109,7.0,5.0,June Wins!
3,269,6.0,3.0,June Wins!
4,239,6.0,5.0,June Wins!
...,...,...,...,...
593,63,NaN,1.0,nan alert
594,598,NaN,1.0,nan alert
595,487,NaN,1.0,nan alert
596,555,NaN,1.0,nan alert


In [67]:
final_data['result'].unique()

array(["It's a tie!", 'June Wins!', 'May Wins!', 'nan alert'],
      dtype=object)